In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('bank/bank-full.csv',sep=';')
df.head()

In [ ]:
#get unique values in each column
for col in df.columns:
    print(col, df[col].unique())
    

In [ ]:
#remove rows with unknown values
df = df[(df['job'] != 'unknown') & (df['marital'] != 'unknown') & (df['education'] != 'unknown') & (df['default'] != 'unknown') & (df['housing'] != 'unknown') & (df['loan'] != 'unknown') & (df['contact'] != 'unknown') & (df['poutcome'] != 'unknown')]

In [ ]:
for col in df.columns:
    print(col, df[col].value_counts())

In [ ]:
#drop multiple columns
df = df.drop(['balance','day','pdays','duration'],axis=1)
df.head()

In [ ]:
df.shape

In [ ]:
#re index the dataframe
df = df.reset_index(drop=True)
df.head()

In [ ]:
#use label encoder to convert categorical values to numerical values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in ['job','marital','education','default','housing','loan','contact','poutcome','y']:
    df[col] = le.fit_transform(df[col])
df.head()


In [ ]:
#map month to numerical values
month_map = {'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12}
df['month'] = df['month'].map(month_map)
df.head()

In [ ]:
for col in df.columns:
    print(col, df[col].unique())

In [ ]:
sample_dict = {}
for i in range (10):
    sample_dict['S'+str(i+1)] = df.sample(frac=0.25,replace=False)

sample_dict

In [ ]:
#import svm
#over sampler
from imblearn.over_sampling import SMOTE

import random
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
kernel_list = ['linear','poly','rbf','sigmoid']
best_kernel = ''
best_accuracy = 0
best_C = 0
best_gamma = 0
result_dict = {}

for key, value in sample_dict.items():
    sample_df = value
    X = sample_df.drop('y',axis=1)
    feature_name = X.columns
    y = sample_df['y']
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
    sm = SMOTE(random_state=2)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    for iter in range(1000):
        kernel = random.choice(kernel_list)
       
        C = random.uniform(0,1)
        gamma = random.uniform(0,1)
        model = SVC(kernel=kernel,C=C,gamma=gamma,max_iter=2)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test,y_pred)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_kernel = kernel
            best_C = C
            best_gamma = gamma
    result_dict[key] = {'accuracy' : best_accuracy, 'kernel' : best_kernel, 'C' : best_C, 'gamma' : best_gamma}
    print('Best accuracy for sample ',key,' is ',best_accuracy)


    

In [ ]:
print(result_dict)

In [ ]:
#convert dict to dataframe
result_df = pd.DataFrame(result_dict)
result_df = result_df.transpose()
result_df

In [ ]:
max_accuracy = result_df['accuracy'].max()
best_sample_key = result_df[result_df['accuracy'] == max_accuracy].index[0]
best_sample_key

In [ ]:
best_sample = sample_dict[best_sample_key]
best_sample.head()

In [ ]:
kernel_list = ['linear','poly','rbf','sigmoid']
accuracy_list = []
best_accuracy = 0



X = sample_df.drop('y',axis=1)
feature_name = X.columns
y = sample_df['y']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
sm = SMOTE(random_state=2)
X_train, y_train = sm.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

for iter in range(1000):
    kernel = random.choice(kernel_list)
    
    C = random.uniform(0,1)
    gamma = random.uniform(0,1)
    model = SVC(kernel=kernel,C=C,gamma=gamma,max_iter=2)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
    accuracy_list.append(best_accuracy)
        
   

In [ ]:
import matplotlib.pyplot as plt
plt.plot(accuracy_list)